# Denison CS-181/DA-210 Homework

---

## Normalization to Tidy Data

In [1]:
import os
import io
import sys
import re
import pandas as pd

from contextlib import redirect_stdout

def add_modules():
    """
    Starting at the current directory and proceeding up the file system
    tree, search for a directory named `modules`.  If found, and if not
    already there, add to the Python module search path.
    
    Params: None
    
    Return: None
    """
    directory = "."
    levels = 0
    while not os.path.isdir(os.path.join(directory, "modules")) and \
          levels < 5:
        directory = os.path.join(directory, "..")
        levels += 1
    module_path = os.path.abspath(os.path.join(directory, "modules"))
    if os.path.isdir(module_path):
        if not module_path in sys.path:
            sys.path.append(module_path)

add_modules()
import util

datadir = util.resolve_dir("tabulardata")

In [2]:
# Source Data Frame Generation

students = pd.read_csv(os.path.join(datadir, "students.csv"))
majors = pd.read_csv(os.path.join(datadir, "majors.csv"))
student_majors0 = pd.merge(students, majors, how="left", on="majorid")
student_majors1 = student_majors0[['studentid', 'studentlast', 'studentfirst', 'majorid', 'majorname', 'departmentid']]
student_majors2 = student_majors1.sort_values(['studentlast', 'studentfirst'])
student_majors = student_majors2.reset_index(drop=True)

tourism = pd.read_csv(os.path.join(datadir, "tourism.csv"), index_col="date")
tourism = tourism[['visitors', 'roomsavail', 'avgrate', 'gamingrevenue']]

path = os.path.join(datadir, "topnames.csv")
topnames = pd.read_csv(path)
topnames.set_index(['year', 'sex'], inplace=True)
topnames.sort_index(level='year', ascending=False, inplace=True)
topnames0 = topnames.reset_index(drop=False)

df5 = topnames0.pivot(index='year', columns='sex')
df5.columns = ['FemaleName', 'MaleName', 'FemaleCount', 'MaleCount']
df6 = df5.loc[:,['FemaleName', 'FemaleCount', 'MaleName', 'MaleCount']]
valuevar_cols = df6.sort_index(axis=0, ascending=False)

df = topnames0.drop(axis=1, columns=['count'])
df2 = df.pivot(index='year', columns='sex')
df3 = df2.reset_index()
df3.columns = ['year', 'Female', 'Male']
df4 = df3.set_index('year')
namesbyyear = df4.T.loc[:,2014:].copy()
namesbyyear.index = pd.Index(['Female', 'Male'], name='sex')
namesbyyear.columns = [2014, 2015, 2016, 2017, 2018]

counts = topnames0.drop(axis=1, columns=['name'])
counts2 = counts.pivot(index='year', columns='sex')
counts3 = counts2.reset_index()
counts3.columns = ['year', 'Female', 'Male']
counts4 = counts3.set_index('year')
countsbyyear = counts4.T.loc[:,2014:].copy()
countsbyyear.index = pd.Index(['Female', 'Male'], name='sex')
countsbyyear.columns = [2014, 2015, 2016, 2017, 2018]

**Q** Consider the `student_majors` data frame shown below.  In the Markdown solution cell, write down the one or more functional dependencies entailed in the data set.  ... this data set should be familiar to you from your homework.

Then write the English steps to make this into one or more tidy data tables.

Finally, in the code cell, translate your steps into actual code.  If you find that you need to change your steps based on writing and debugging the actual code, make those changes.  This will help you to learn to be clear, precise, and unambiguous in your English-based algorithm.

In [6]:
student_majors.head()

,studentid,studentlast,studentfirst,majorid,majorname,departmentid
0,61906,Adams,Aaron,CS,Computer Science,MATH
1,63100,Adams,Abigail,MUS,Music,MUS
2,61961,Adams,Jeremy,BIOL,Biology,BIOL
3,63766,Adams,Raymond,NaN,NaN,NaN
4,63222,Adams,Sandra,NaN,NaN,NaN


Functional Dependencies: studentid, majorid

Steps:
1. Split into two tables, one with studentid, studentlast, and studentfirst and the other table have majorid, majorname, and departmentid
2. get ride of empty rows in majors table
2. Pivot the students table and the majors table

In [28]:
students = student_majors[['studentid','studentlast','studentfirst']]
majors = student_majors[['majorid', 'majorname','departmentid']]

students = students.set_index('studentid')
majors = majors.set_index('majorid')

print(students.head())
print(majors.head())

AttributeError: 'majorid' is not a valid function for 'Series' object

**Q** Consider the `tourism` data frame, representing actual data from the Las Vegas tourism bureau.  In the Markdown cell, and perhaps consulting section 9.4.1, assess the table for tidy and, if not tidy, give the functional dependency, and an enumerated list of the steps required to make it tidy.  Then, in the following code cell, write the code to make it happen.

In [10]:
tourism.head()

,visitors,roomsavail,avgrate,gamingrevenue
date,,,,
Jan-2017,3508800,148693,149.13,926124000.0
Feb-2017,3150500,147866,116.73,825618000.0
Mar-2017,3783900,147717,148.55,857317000.0
Apr-2017,3545100,147566,123.52,763254000.0
May-2017,3602100,147656,124.51,860700000.0


Functional Dependency: visitors

1. split up date into month and year and delete date
2. pivot

In [24]:
tourism_tidy = tourism.copy()
tourism_tidy = tourism_tidy.reset_index()
tourism_tidy['month'] = tourism_tidy['date'].apply(lambda s: s[:3])
tourism_tidy['year'] = tourism_tidy['date'].apply(lambda s: s[-4:])
del tourism_tidy['date']
tourism_tidy = tourism_tidy.set_index(['month','year'])
tourism_tidy.head(14)

,,visitors,roomsavail,avgrate,gamingrevenue
month,year,,,,
Jan,2017,3508800,148693,149.13,926124000.0
Feb,2017,3150500,147866,116.73,825618000.0
Mar,2017,3783900,147717,148.55,857317000.0
Apr,2017,3545100,147566,123.52,763254000.0
May,2017,3602100,147656,124.51,860700000.0
Jun,2017,3605700,147208,118.95,764615000.0
Jul,2017,3788800,147013,124.48,848573000.0
Aug,2017,3562900,146847,113.25,841591000.0
Sep,2017,3566700,146847,139.29,830810000.0


**Q** Consider the `valuevar_cols` data frame.  This is the same one we consdered in class.  In the Markdown cell, assess the data for tidy (which TidyData constraints are violated, and what red flags do you see), and write the FD for the data set.

In class, a solution path was suggested that "split" the composite table into two tables, one for male and the other for female, work on the individual tables, and then combine them to get tidy data.  In the Markdown cell, make this set of steps precise and unambiguous.  

Then, in the code cell, enter your code solution.

In [ ]:
valuevar_cols.head(8)

YOUR ANSWER HERE

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
topnames_tidy.head(10)

**Q** Similar to the last question, in class we showed the two tables `namesbyyear` and `countsbyyear` and the steps to tidy in class involved first combining the tables and then doing a melt.

Your goal in this question is to come up with a different solution wherein you first melt the two tables and then combine them, but making sure to incorporate the names and counts into the tables so that, in the end, we have columns for sex, year, name, and count.

You are only requested to do the code for this question.

In [ ]:
namesbyyear

In [ ]:
countsbyyear

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()
namecounts